# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 31

Group Members:
- Muhammad Fadhil Atha (18221003)
- Nicholas Francis Aditjandra (18221005)
- Hanan Fitra Salam (18222133)
- Salsabila Azzahra (18222139)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib
from google.colab import drive
from collections import defaultdict

## Import Dataset

In [ ]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public
#https://drive.google.com/file/d/1RAYQm6EeBllh7b1Ul-SBke9XKTxvTE_Z/view?usp=sharing
df = pd.read_csv('https://drive.google.com/uc?id=1RAYQm6EeBllh7b1Ul-SBke9XKTxvTE_Z',on_bad_lines='skip')
print(df.isnull().sum())


id                                0
FILENAME                      57532
URL                           43487
URLLength                     60639
Domain                        70197
DomainLength                  46319
IsDomainIP                    42130
TLD                           45399
CharContinuationRate          48042
TLDLegitimateProb             52873
URLCharProb                   52071
TLDLength                     47731
NoOfSubDomain                 44060
HasObfuscation                65720
NoOfObfuscatedChar            66798
ObfuscationRatio              64598
NoOfLettersInURL              63338
LetterRatioInURL              65746
NoOfDegitsInURL               58810
DegitRatioInURL               53508
NoOfEqualsInURL               61578
NoOfQMarkInURL                44101
NoOfAmpersandInURL            45387
NoOfOtherSpecialCharsInURL    47629
SpacialCharRatioInURL         62834
IsHTTPS                       49362
LineOfCode                    69153
LargestLineLength           

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [ ]:
X = df.drop(columns=['label'])  # Features
y = df['label']
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # Use stratify if classification
)
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)

Training data shape: (112323, 55)
Validation data shape: (28081, 55)


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [ ]:
numeric_columns = X_train.select_dtypes(include=['float64']).columns


for col in numeric_columns:
    mean_value = X_train[col].mean()
    X_train[col].fillna(mean_value, inplace=True)
    X_val[col].fillna(mean_value, inplace=True)


object_columns = X_train.select_dtypes(include=['object']).columns

for col in object_columns:
    most_frequent = X_train[col].mode()[0]
    X_train[col].fillna(most_frequent, inplace=True)
    X_val[col].fillna(most_frequent, inplace=True)


print("\nCleaned Training Data (X_train):")
print(X_train.head())
print("\nCleaned Validation Data (X_val):")
print(X_val.head())


Cleaned Training Data (X_train):
            id  FILENAME    URL  URLLength  Domain  DomainLength  IsDomainIP  \
135080  226786     17522  67930  27.067934    1111     19.612679    0.000000   
97659   163910         0  10621  27.642457    5948     19.612679    0.000000   
11488    19126     57069    624  27.642457    1111     19.612679    0.000000   
67815   113381     55321   3201  27.642457    1111     19.612679    0.000433   
97148   163038     48043  44448  27.067934    1111     17.000000    0.000000   

        TLD  CharContinuationRate  TLDLegitimateProb  ...      Bank       Pay  \
135080  117              0.918147           0.522907  ...  0.172711  0.344851   
97659   117              1.000000           0.277364  ...  0.000000  0.344851   
11488   117              1.000000           0.522907  ...  0.172711  0.344851   
67815   117              0.918147           0.079963  ...  0.000000  0.000000   
97148   117              0.918147           0.522907  ...  0.172711  0.344851   

<ipython-input-100-79780a8419ac>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(mean_value, inplace=True)
<ipython-input-100-79780a8419ac>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [ ]:
def replace_outliers_with_mean(df, column):

    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1


    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    mean_value = df[column].mean()
    df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound), mean_value, df[column])

    return df[column]


numeric_columns = X_train.select_dtypes(include=['float64']).columns


for col in numeric_columns:
    X_train[col] = replace_outliers_with_mean(X_train, col)


for col in numeric_columns:
    X_val[col] = replace_outliers_with_mean(X_train, col)


print("\nCleaned Training Data (X_train):")
print(X_train.head())

print("\nCleaned Validation Data (X_val):")
print(X_val.head())



Cleaned Training Data (X_train):
            id  FILENAME    URL  URLLength  Domain  DomainLength  IsDomainIP  \
135080  226786     17522  67930  27.067934    1111     19.612679    0.000000   
97659   163910         0  10621  27.642457    5948     19.612679    0.000000   
11488    19126     57069    624  27.642457    1111     19.612679    0.000000   
67815   113381     55321   3201  27.642457    1111     19.612679    0.000433   
97148   163038     48043  44448  27.067934    1111     17.000000    0.000000   

        TLD  CharContinuationRate  TLDLegitimateProb  ...      Bank       Pay  \
135080  117              0.918147           0.522907  ...  0.172711  0.344851   
97659   117              1.000000           0.277364  ...  0.000000  0.344851   
11488   117              1.000000           0.522907  ...  0.172711  0.344851   
67815   117              0.918147           0.079963  ...  0.000000  0.000000   
97148   117              0.918147           0.522907  ...  0.172711  0.344851   

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
X_train.drop_duplicates()
X_val.drop_duplicates()
print("\nCleaned Training Data (X_train):")
print(X_train.head())

print("\nCleaned Validation Data (X_val):")
print(X_val.head())


Cleaned Training Data (X_train):
            id  FILENAME    URL  URLLength  Domain  DomainLength  IsDomainIP  \
135080  226786     17522  67930  27.067934    1111     19.612679    0.000000   
97659   163910         0  10621  27.642457    5948     19.612679    0.000000   
11488    19126     57069    624  27.642457    1111     19.612679    0.000000   
67815   113381     55321   3201  27.642457    1111     19.612679    0.000433   
97148   163038     48043  44448  27.067934    1111     17.000000    0.000000   

        TLD  CharContinuationRate  TLDLegitimateProb  ...      Bank       Pay  \
135080  117              0.918147           0.522907  ...  0.172711  0.344851   
97659   117              1.000000           0.277364  ...  0.000000  0.344851   
11488   117              1.000000           0.522907  ...  0.172711  0.344851   
67815   117              0.918147           0.079963  ...  0.000000  0.000000   
97148   117              0.918147           0.522907  ...  0.172711  0.344851   

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [ ]:
columns_to_drop = ['FILENAME', 'URL', 'Domain', 'TLD', 'id']

X_train = X_train.drop(columns=columns_to_drop, errors='ignore')
X_val = X_val.drop(columns=columns_to_drop, errors='ignore')

print("Selected Features:")
print(X_train.columns.tolist())
print(X_val.columns.tolist())


Selected Features:
['URLLength', 'DomainLength', 'IsDomainIP', 'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength', 'HasTitle', 'Title', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon', 'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']
['URLLength', 'DomainLength', 'IsDomainIP', 'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDoma

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [ ]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class FeatureEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, fill_value='missing', one_hot_columns=None):
        self.fill_value = fill_value
        self.one_hot_columns = one_hot_columns
        self._one_hot_encoders = {}

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        if self.one_hot_columns is None:
            self.one_hot_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
        for col in self.one_hot_columns:
            self._one_hot_encoders[col] = X[col].fillna(self.fill_value).unique()
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        X_encoded = X.copy()
        X_encoded.fillna(self.fill_value, inplace=True)
        for col in self.one_hot_columns:
            if col in X_encoded:
                one_hot_df = pd.get_dummies(
                    X_encoded[col],
                    prefix=col,
                    drop_first=False,
                    dtype=int
                )
                X_encoded = pd.concat([X_encoded.drop(columns=[col]), one_hot_df], axis=1)
        return X_encoded

df = pd.read_csv('https://drive.google.com/uc?id=1RAYQm6EeBllh7b1Ul-SBke9XKTxvTE_Z', on_bad_lines='skip')
data = df[['TLD', 'Robots']]

print(data.head())

encoder = FeatureEncoder(fill_value=0)
encoder.fit(data)
transformed_data = encoder.transform(data)

print(transformed_data.head())

   TLD  Robots
0  NaN     0.0
1   to     NaN
2   au     0.0
3  com     0.0
4  NaN     1.0
   Robots  TLD_0  TLD_108  TLD_12  TLD_121  TLD_128  TLD_13  TLD_134  TLD_140  \
0     0.0      1        0       0        0        0       0        0        0   
1     0.0      0        0       0        0        0       0        0        0   
2     0.0      0        0       0        0        0       0        0        0   
3     0.0      0        0       0        0        0       0        0        0   
4     1.0      1        0       0        0        0       0        0        0   

   TLD_146  ...  TLD_wtf  TLD_xn--p1ai  TLD_xyz  TLD_ye  TLD_yoga  \
0        0  ...        0             0        0       0         0   
1        0  ...        0             0        0       0         0   
2        0  ...        0             0        0       0         0   
3        0  ...        0             0        0       0         0   
4        0  ...        0             0        0       0         0   

   TLD_y

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaler=MinMaxScaler()):
        self.scaler = scaler

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        numerical_columns = X.select_dtypes(include=['float64']).columns
        self.scaler.fit(X[numerical_columns])
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        numerical_columns = X.select_dtypes(include=['float64']).columns
        return self.scaler.transform(X[numerical_columns])

df = pd.read_csv('https://drive.google.com/uc?id=1RAYQm6EeBllh7b1Ul-SBke9XKTxvTE_Z', on_bad_lines='skip')

X = df.drop(columns=['label'])
y = df['label']

print("Data Awal:")
print(X.head())

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

numerical_columns = X_train.select_dtypes(include=['float64']).columns
min_max_scaler = FeatureScaler()
X_train[numerical_columns] = min_max_scaler.fit_transform(X_train[numerical_columns])

# numerical_columns = df.select_dtypes(include=['float64']).columns
# min_max_scaler = FeatureScaler()
# df[numerical_columns] = min_max_scaler.fit_transform(df[numerical_columns])

print("\nMin-Max Scaled Training Data:")
print(X_train.head())

Data Awal:
   id     FILENAME                                   URL  URLLength  \
0   1          NaN             https://www.northcm.ac.th       24.0   
1   4  8135291.txt                   http://uqr.to/1il1z        NaN   
2   5   586561.txt  https://www.woolworthsrewards.com.au       35.0   
3   6          NaN                                   NaN       31.0   
4  11   412632.txt                                   NaN        NaN   

                         Domain  DomainLength  IsDomainIP  TLD  \
0             www.northcm.ac.th          17.0         0.0  NaN   
1                           NaN           NaN         NaN   to   
2  www.woolworthsrewards.com.au          28.0         0.0   au   
3                           NaN           NaN         NaN  com   
4        www.nyprowrestling.com          22.0         0.0  NaN   

   CharContinuationRate  TLDLegitimateProb  ...  Bank  Pay  Crypto  \
0              0.800000                NaN  ...   NaN  0.0     0.0   
1              1.000000  

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class FeatureEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, fill_value='missing', one_hot_columns=None):
        self.fill_value = fill_value
        self.one_hot_columns = one_hot_columns
        self._one_hot_encoders = {}

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        if self.one_hot_columns is None:
            self.one_hot_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
        for col in self.one_hot_columns:
            self._one_hot_encoders[col] = X[col].fillna(self.fill_value).unique()
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        X_encoded = X.copy()
        X_encoded.fillna(self.fill_value, inplace=True)
        for col in self.one_hot_columns:
            if col in X_encoded:
                one_hot_df = pd.get_dummies(
                    X_encoded[col],
                    prefix=col,
                    drop_first=False,
                    dtype=int
                )
                X_encoded = pd.concat([X_encoded.drop(columns=[col]), one_hot_df], axis=1)
        return X_encoded

df = pd.read_csv('https://drive.google.com/uc?id=1RAYQm6EeBllh7b1Ul-SBke9XKTxvTE_Z', on_bad_lines='skip')
data = df[['TLD', 'Robots']]

print(data.head())

encoder = FeatureEncoder(fill_value=0)
encoder.fit(data)
transformed_data = encoder.transform(data)

print(transformed_data.head())

   TLD  Robots
0  NaN     0.0
1   to     NaN
2   au     0.0
3  com     0.0
4  NaN     1.0
   Robots  TLD_0  TLD_108  TLD_12  TLD_121  TLD_128  TLD_13  TLD_134  TLD_140  \
0     0.0      1        0       0        0        0       0        0        0   
1     0.0      0        0       0        0        0       0        0        0   
2     0.0      0        0       0        0        0       0        0        0   
3     0.0      0        0       0        0        0       0        0        0   
4     1.0      1        0       0        0        0       0        0        0   

   TLD_146  ...  TLD_wtf  TLD_xn--p1ai  TLD_xyz  TLD_ye  TLD_yoga  \
0        0  ...        0             0        0       0         0   
1        0  ...        0             0        0       0         0   
2        0  ...        0             0        0       0         0   
3        0  ...        0             0        0       0         0   
4        0  ...        0             0        0       0         0   

   TLD_y

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import pandas as pd

data = pd.read_csv('https://drive.google.com/uc?id=1RAYQm6EeBllh7b1Ul-SBke9XKTxvTE_Z', on_bad_lines='skip')
df = data[['label', 'URL', 'URLLength']]

X = df.drop(columns=['label'])
y = df['label']

df_balanced = pd.concat([X, y], axis=1)

majority_class = df_balanced[df_balanced['label'] == 1]
minority_class = df_balanced[df_balanced['label'] == 0]


majority_class_downsampled = resample(majority_class,
                                      replace=False,
                                      n_samples=len(minority_class),
                                      random_state=42)

df_balanced = pd.concat([majority_class_downsampled, minority_class])

df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

X_balanced = df_balanced.drop(columns=['label'])
y_balanced = df_balanced['label']

print(f"Class Distribution:\n{y.value_counts()}")
print(f"\nResampled Class Distribution:\n{y_balanced.value_counts()}")

Class Distribution:
label
1    129850
0     10554
Name: count, dtype: int64

Resampled Class Distribution:
label
1    10554
0    10554
Name: count, dtype: int64


# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [ ]:
# from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEncoder(BaseEstimator, TransformerMixin):
# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [ ]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaler=MinMaxScaler()):
        self.scaler = scaler

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        numerical_columns = X.select_dtypes(include=['float64']).columns
        self.scaler.fit(X[numerical_columns])
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        numerical_columns = X.select_dtypes(include=['float64']).columns
        return self.scaler.transform(X[numerical_columns])

class FeatureEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, fill_value='missing', one_hot_columns=None):
        self.fill_value = fill_value
        self.one_hot_columns = one_hot_columns
        self._one_hot_encoders = {}

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        if self.one_hot_columns is None:
            self.one_hot_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
        for col in self.one_hot_columns:
            self._one_hot_encoders[col] = X[col].fillna(self.fill_value).unique()
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        X_encoded = X.copy()
        for col in self.one_hot_columns:
            if col in X_encoded:
                X_encoded[col] = X_encoded[col].fillna(self.fill_value).astype("object")
        for col in self.one_hot_columns:
            if col in X_encoded:
                one_hot_df = pd.get_dummies(
                    X_encoded[col],
                    prefix=col,
                    drop_first=False,
                    dtype=int
                )
                X_encoded = pd.concat([X_encoded.drop(columns=[col]), one_hot_df], axis=1)
        return X_encoded

df = pd.read_csv('https://drive.google.com/uc?id=1RAYQm6EeBllh7b1Ul-SBke9XKTxvTE_Z', on_bad_lines='skip')

X = df.drop(columns=['label'])
y = df['label']

pipe = Pipeline([("scaler", FeatureScaler()),
                 ("encoder", FeatureEncoder(fill_value=0))])

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

train_set = pipe.fit_transform(X_train)

val_set = pipe.transform(X_val)

print("Training data shape:", train_set.shape)
print(train_set.head())
print("\nValidation data shape:", val_set.shape)
print(val_set.head())

Training data shape: (112323, 49)
         0         1    2     3        4         5         6         7    8   \
0  0.002891       NaN  0.0   NaN  1.00000       NaN       NaN  0.166667  0.0   
1  0.005493       NaN  0.0  1.00      NaN  0.645354       NaN  0.166667  0.0   
2       NaN       NaN  0.0  1.00  1.00000  0.716760       NaN  0.166667  NaN   
3       NaN  0.280899  NaN  0.56  0.15292  0.643877  0.090909  0.333333  NaN   
4  0.002891  0.146067  0.0   NaN  1.00000  0.755511  0.090909  0.166667  NaN   

    9   ...   39   40   41   42   43        44        45        46        47  \
0  NaN  ...  NaN  NaN  NaN  1.0  NaN       NaN  0.002475       NaN  0.000205   
1  0.0  ...  0.0  1.0  0.0  0.0  NaN  0.003842       NaN  0.005246  0.000000   
2  NaN  ...  1.0  NaN  0.0  1.0  NaN       NaN  0.012376  0.007244       NaN   
3  NaN  ...  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000  0.000000       NaN   
4  NaN  ...  NaN  1.0  NaN  1.0  NaN       NaN       NaN  0.011590       NaN   

    

/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [ ]:
import numpy as np

class KNNFromScratch:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def manhattan_distance(self, x1, x2):
        return np.sum(np.abs(x1 - x2))

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

        self.X_train_numeric = X_train.select_dtypes(include=['float64', 'int64'])

    def predict(self, X_test):
        predictions = []

        X_test_numeric = X_test.select_dtypes(include=['float64', 'int64'])

        for test_point in X_test_numeric.values:
            distances = []
            for i, train_point in enumerate(self.X_train_numeric.values):
                if self.distance_metric == 'euclidean':
                    distance = self.euclidean_distance(test_point, train_point)
                elif self.distance_metric == 'manhattan':
                    distance = self.manhattan_distance(test_point, train_point)
                distances.append((distance, i))

            distances.sort(key=lambda x: x[0])
            k_neighbors = [self.y_train.iloc[i] for _, i in distances[:self.k]]

            prediction = max(set(k_neighbors), key=k_neighbors.count)
            predictions.append(prediction)

        return np.array(predictions)

## Scratch

In [ ]:
knn_scratch = KNNFromScratch(k=3, distance_metric='euclidean')

knn_scratch.fit(X_balanced, y_balanced)

y_pred_knn_scratch = knn_scratch.predict(X_val)

from sklearn.metrics import accuracy_score, classification_report

print("Accuracy of KNN from Scratch:", accuracy_score(y_val, y_pred_knn_scratch))
print("\nClassification Report for KNN from Scratch:\n", classification_report(y_val, y_pred_knn_scratch))

## Scikit-learn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

knn_sklearn = KNeighborsClassifier(n_neighbors=3)

knn_sklearn.fit(X_balanced, y_balanced)

y_pred_knn_sklearn = knn_sklearn.predict(X_val)

print("Accuracy of KNN (scikit-learn):", accuracy_score(y_val, y_pred_knn_sklearn))
print("\nClassification Report for KNN (scikit-learn):\n", classification_report(y_val, y_pred_knn_sklearn))


## B. Naive Bayes

In [ ]:
drive.mount('/content/drive')
def encode_categorical_columns(df, encoders=None):
    """
    Encode categorical columns using LabelEncoder.
    If encoders are provided, use them; otherwise, create new encoders.
    """
    categorical_columns = df.select_dtypes(include=['object']).columns
    encoders = encoders or {}
    for col in categorical_columns:
        if col not in encoders:
            encoders[col] = LabelEncoder()
            df[col] = encoders[col].fit_transform(df[col].astype(str))
        else:
            df[col] = encoders[col].transform(df[col].astype(str))
    return df, encoders

X_train, label_encoders = encode_categorical_columns(X_train)
X_val, _ = encode_categorical_columns(X_val, encoders=label_encoders)

model = GaussianNB()
model.fit(X_train, y_train)

model_path = "/content/drive/My Drive/TUBES 2 AI/Model/naive_bayes_model.pkl"
encoders_path = "/content/drive/My Drive/TUBES 2 AI/Model/label_encoders.pkl"
joblib.dump(model, model_path)
joblib.dump(label_encoders, encoders_path)
print(f"Model saved to: {model_path}")
print(f"Encoders saved to: {encoders_path}")

loaded_model = joblib.load(model_path)
loaded_encoders = joblib.load(encoders_path)
print("Model and encoders loaded successfully.")

y_pred = loaded_model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))

# Scratch
# drive.mount('/content/drive')


# class GaussianNaiveBayes:
#     def __init__(self):
#         self.classes = None
#         self.priors = {}
#         self.means = {}
#         self.variances = {}

#     def fit(self, X, y):
#         """
#         Train the Gaussian Naive Bayes model.
#         """
#         self.classes = np.unique(y)
#         for cls in self.classes:
#             X_cls = X[y == cls]
#             self.priors[cls] = len(X_cls) / len(X)
#             self.means[cls] = np.mean(X_cls, axis=0)
#             self.variances[cls] = np.var(X_cls, axis=0)

#     def calculate_likelihood(self, x, mean, var):
#         """
#         Calculate the Gaussian likelihood of x given mean and variance.
#         """
#         eps = 1e-9  # To prevent division by zero
#         coeff = 1 / np.sqrt(2 * np.pi * (var + eps))
#         exponent = np.exp(-((x - mean) ** 2) / (2 * (var + eps)))
#         return coeff * exponent

#     def predict_single(self, x):
#         """
#         Predict the class for a single sample.
#         """
#         posteriors = {}
#         for cls in self.classes:
#             prior = np.log(self.priors[cls])
#             likelihood = np.sum(np.log(self.calculate_likelihood(x, self.means[cls], self.variances[cls])))
#             posteriors[cls] = prior + likelihood
#         return max(posteriors, key=posteriors.get)

#     def predict(self, X):
#         """
#         Predict classes for a dataset.
#         """
#         return np.array([self.predict_single(x) for x in X])


# def ensure_numeric(X):
#     """
#     Convert non-numeric columns to numeric or drop them.
#     """
#     numeric_X = X.copy()
#     for col in X.columns:
#         if X[col].dtype not in ['int64', 'float64']:
#             numeric_X[col] = pd.to_numeric(X[col], errors='coerce')
#     return numeric_X.dropna(axis=1)


# X_train = ensure_numeric(pd.DataFrame(X_train))
# X_val = ensure_numeric(pd.DataFrame(X_val))


# X_train = X_train.values
# X_val = X_val.values


# model = GaussianNaiveBayes()
# model.fit(X_train, y_train)


# y_pred = model.predict(X_val)


# def accuracy_score_manual(y_true, y_pred):
#     correct = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
#     return correct / len(y_true)

# def classification_report_manual(y_true, y_pred):
#     report = defaultdict(dict)
#     classes = np.unique(y_true)

#     for cls in classes:
#         tp = sum((y_true == cls) & (y_pred == cls))
#         fp = sum((y_true != cls) & (y_pred == cls))
#         fn = sum((y_true == cls) & (y_pred != cls))
#         tn = sum((y_true != cls) & (y_pred != cls))
#         support = tp + fn

#         precision = tp / (tp + fp) if (tp + fp) > 0 else 0
#         recall = tp / (tp + fn) if (tp + fn) > 0 else 0
#         f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

#         report[cls] = {
#             'precision': precision,
#             'recall': recall,
#             'f1-score': f1_score,
#             'support': support
#         }

#     return report

# accuracy = accuracy_score_manual(y_val, y_pred)
# report = classification_report_manual(y_val, y_pred)


# output_path = '/content/drive/My Drive/TUBES 2 AI/Model/gaussian_naive_bayes_model.joblib'
# joblib.dump({'model': model, 'accuracy': accuracy, 'classification_report': report}, output_path)


# loaded_data = joblib.load(output_path)
# loaded_model = loaded_data['model']
# loaded_accuracy = loaded_data['accuracy']
# loaded_report = loaded_data['classification_report']


# print("Model and results saved to:", output_path)
# print("Accuracy:", loaded_accuracy)
# for cls, metrics in loaded_report.items():
#     print(f"Class {cls}: {metrics}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to: /content/drive/My Drive/TUBES 2 AI/Model/naive_bayes_model.pkl
Encoders saved to: /content/drive/My Drive/TUBES 2 AI/Model/label_encoders.pkl
Model and encoders loaded successfully.
Accuracy: 0.9248246145080303

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      2111
           1       0.92      1.00      0.96     25970

    accuracy                           0.92     28081
   macro avg       0.46      0.50      0.48     28081
weighted avg       0.86      0.92      0.89     28081



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [ ]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`